<a href="https://colab.research.google.com/github/34-anish/FOML/blob/main/Question_No_2c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mord


[notice] A new release of pip is available: 23.1.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Data Preprocessing

Since the quality is already in its natural order, the encoding is pretty straightforward approach to bring down the quality from `3->9 ` to `0->6`

In [ ]:
import pandas as pd
url = "winequality-white.csv"
wine_data = pd.read_csv(url, sep=";")

from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
wine_data[["quality"]] = enc.fit_transform(wine_data[["quality"]]).astype(int)

In [ ]:
wine_data.sample(n=5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
829,7.1,0.16,0.36,1.2,0.043,21.0,90.0,0.99250,3.16,0.42,11.0,4
3447,6.0,0.17,0.29,9.7,0.044,33.0,98.0,0.99536,3.12,0.36,9.2,3
189,6.5,0.28,0.28,8.5,0.047,54.0,210.0,0.99620,3.09,0.54,8.9,1
978,7.7,0.28,0.33,6.7,0.037,32.0,155.0,0.99510,3.39,0.62,10.7,4
1374,6.9,0.23,0.34,2.7,0.032,24.0,121.0,0.99020,3.14,0.38,12.4,4


## Data Splitting

The 80-20 split is performed in the dataset.

In [ ]:
from sklearn.model_selection import train_test_split

X = wine_data.drop("quality", axis=1)
y = wine_data["quality"]

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Normalizing data

Here, we perform normalization on the x values so as to make sure that features come to same scales. Also since we are considering all 11 features as x vector, it is supposed to make convergence faster.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_normalized = scaler.fit_transform(x_train)
x_test_normalized = scaler.transform(x_test)

## Fitting the model

We use [mord](https://pythonhosted.org/mord/) library, which has **OrdinalRidge** as the *Regression-based model* for ordinal regression task.

In [ ]:
assert(len(x_train_normalized) == len(y_train))

from mord import OrdinalRidge

model = OrdinalRidge()
model.fit(x_train_normalized, y_train)
y_hat = model.predict(x_test_normalized)

## Evaluation Metrics


We have utilized two evaluation metrics which quantify the accuracy of predictive ordinal scales `{ ̂y1,...,ŷt}` with respect to true targets `{y1,...,yt}`:


*   ***Mean absolute error*** is the average deviation of the prediction from the true target, in which we treat the ordinal scales as consecutive integers;
*   ***Mean zero-one error*** gives an error of 1 to every incorrect prediction that is the fraction of incorrect predictions.

Our objective is to minimize both the metric.



In [ ]:
from sklearn.metrics import mean_absolute_error

def mean_zo_error(y_true, y_pred):
  n = len(y_true)
  error_count = sum(int(y_hat != y) for y_hat, y in zip(y_pred, y_true))
  mean_zo_error = error_count / n
  return mean_zo_error

mae = mean_absolute_error(y_test, y_hat)
mzo_error = mean_zo_error(y_test, y_hat)

print(mae, mzo_error)

0.5408163265306123 0.4857142857142857


## Comparision with Linear and Logistic Regression

In [ ]:
from sklearn.linear_model import LinearRegression

lin_model = LinearRegression()
lin_model.fit(x_train_normalized, y_train)
y_pred_lin = lin_model.predict(x_test_normalized)

mse_lin = mean_absolute_error(y_test, y_pred_lin)
mzo_error_lin = mean_zo_error(y_test, y_pred_lin)
print(mse_lin, mzo_error_lin)


0.5862665383250467 1.0


We see that mean zero one error for linear regression is very high (i.e. almost close to 1 or maybe 1 depending upon test_data), so the predicted quality does not fall in same level as the true quality when we use linear regression. However the MAE does show relatively smaller value, it is not a good metric to be used as **single statistic** for our problem.

In [ ]:
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression()
log_model.fit(x_train_normalized, y_train)
y_pred_log = log_model.predict(x_test_normalized)

mse_log = mean_absolute_error(y_test, y_pred_log)
mzo_error_log = mean_zo_error(y_test, y_pred_log)
print(mse_log, mzo_error_log)

0.539795918367347 0.46938775510204084


C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
